<a href="https://colab.research.google.com/github/Michaelkats5/DemandSync/blob/main/Monte_Carlo_Simulation_for_College_Football_Team_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==================== 1. IMPORTS ====================
import os, glob
import pandas as pd
import numpy as np
from io import StringIO

# ==================== 2. BASE ARRAY (MANUAL DATA) ====================
# Replace this block with your manually curated array
teams_data = [
    {"team": "Texas", "conf": "SEC", "FPI": 24.5, "TalentRank": 4,
     "TalentPoints": 973.54, "ReturningStartersPct": 55.0, "QB_Snaps": 260},
    {"team": "Alabama", "conf": "SEC", "FPI": 20.7, "TalentRank": 2,
     "TalentPoints": 993.55, "ReturningStartersPct": 63.6, "QB_Snaps": 60},
    {"team": "Ohio State", "conf": "Big Ten", "FPI": 20.5, "TalentRank": 3,
     "TalentPoints": 973.69, "ReturningStartersPct": 50.0, "QB_Snaps": 180},
    {"team": "Georgia", "conf": "SEC", "FPI": 19.8, "TalentRank": 1,
     "TalentPoints": 999.9, "ReturningStartersPct": 58.0, "QB_Snaps": 40},
    {"team": "Penn State", "conf": "Big Ten", "FPI": 18.2, "TalentRank": 6,
     "TalentPoints": 950.1, "ReturningStartersPct": 62.0, "QB_Snaps": 150},
    {"team": "Clemson", "conf": "ACC", "FPI": 17.0, "TalentRank": 7,
     "TalentPoints": 940.0, "ReturningStartersPct": 60.0, "QB_Snaps": 220},
    {"team": "Notre Dame", "conf": "Ind", "FPI": 16.5, "TalentRank": 8,
     "TalentPoints": 930.0, "ReturningStartersPct": 61.0, "QB_Snaps": 75},
    {"team": "Oregon", "conf": "Pac-12", "FPI": 15.8, "TalentRank": 5,
     "TalentPoints": 960.0, "ReturningStartersPct": 59.0, "QB_Snaps": 100},
    {"team": "LSU", "conf": "SEC", "FPI": 15.2, "TalentRank": 9,
     "TalentPoints": 925.0, "ReturningStartersPct": 57.0, "QB_Snaps": 210},
    {"team": "Miami", "conf": "ACC", "FPI": 14.9, "TalentRank": 10,
     "TalentPoints": 915.0, "ReturningStartersPct": 56.0, "QB_Snaps": 190},
]
df_array = pd.DataFrame(teams_data)
df_array["team"] = df_array["team"].astype(str).str.strip()
print("Base array loaded:")
print(df_array.head())

# ==================== 3. LOAD TXT FILES ====================
SEARCH_DIRS = [".", "/content", "/content/drive/MyDrive"]
PATTERN = "*_organized_with_coach*.txt"

paths = []
for d in SEARCH_DIRS:
    paths += glob.glob(os.path.join(d, PATTERN))
paths = sorted(set(paths))

print(f"\nFound {len(paths)} text files:")
for p in paths: print(" -", p)

def parse_team_file(path: str) -> pd.DataFrame:
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        text = f.read().replace("\ufeff", "")
    header_key = "team,conference,coach"
    start = text.lower().find(header_key)
    if start == -1:
        raise ValueError(f"No CSV header in {path}")
    csv_block = text[start:]
    df = pd.read_csv(StringIO(csv_block))
    df.columns = [c.strip() for c in df.columns]
    df["team"] = df["team"].astype(str).str.strip()
    rename_map = {
        "conference": "conf",
        "ovr_rating": "OVR",
        "off_rating": "OFF",
        "def_rating": "DEF",
        "qb_avg": "QB_AVG",
        "career_win_pct": "coach_career_win_pct",
    }
    df = df.rename(columns=rename_map)
    keep = [c for c in ["team","conf","coach","coach_career_win_pct","OVR","OFF","DEF","QB_AVG"] if c in df.columns]
    return df[keep]

dfs = []
for p in paths:
    try:
        dfs.append(parse_team_file(p))
        print(f"[OK ] {os.path.basename(p)}")
    except Exception as e:
        print(f"[BAD] {os.path.basename(p)} -> {e}")

teams_extra = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
print("\nExtra data loaded:")
print(teams_extra.head())

# ==================== 4. MERGE MASTER DATA ====================
df = df_array.merge(teams_extra, on="team", how="left")
print("\nMerged dataset preview:")
print(df.head())

# ==================== 5. FAIR RATING CALCULATION ====================
def robust_z(s: pd.Series):
    med = s.median()
    iqr = s.quantile(0.75) - s.quantile(0.25)
    if iqr == 0:
        std = s.std(ddof=0) + 1e-9
        return (s - med) / std
    return (s - med) / (iqr + 1e-9)

numeric = ["FPI","OVR","OFF","DEF","QB_AVG","TalentPoints","ReturningStartersPct","QB_Snaps"]
for c in numeric:
    if c in df.columns:
        df[c+"_rz"] = robust_z(df[c]).clip(-1.5, 1.5)

weights = {
    "FPI_rz": 0.25, "DEF_rz": 0.18, "OFF_rz": 0.14, "OVR_rz": 0.10,
    "QB_AVG_rz": 0.12, "TalentPoints_rz": 0.08, "ReturningStartersPct_rz": 0.07, "QB_Snaps_rz": 0.06
}
weights = {k:v for k,v in weights.items() if k in df.columns}
total = sum(weights.values())
weights = {k: v/total for k,v in weights.items()}

df["feature_score"] = sum(df[k]*w for k,w in weights.items())
df["rating_fair_raw"] = 0.7 * df["feature_score"]
mu, sd = df["rating_fair_raw"].mean(), df["rating_fair_raw"].std(ddof=0)+1e-9
df["rating"] = (df["rating_fair_raw"] - mu)/sd

print("\nTop teams by fair rating:")
print(df[["team","rating"]].sort_values("rating",ascending=False))

# ==================== 6. WIN PROBABILITY FUNCTION ====================
SCALE = 6.0
def win_prob(r1, r2, scale=SCALE):
    return 1 / (1 + np.exp(-(r1-r2)/scale))

print("\nWin probability Texas vs Georgia:", win_prob(
    float(df.loc[df.team=="Texas","rating"]),
    float(df.loc[df.team=="Georgia","rating"])
))


In [ ]:
import glob, os

# Look for all your *_organized_with_coach*.txt files
files = glob.glob("*_organized_with_coach*.txt")
print(f"Found {len(files)} files:")
for f in files:
    print(" -", f)

# Show the first 30 lines of each file
for f in files:
    print("\n" + "="*40)
    print(f"FILE: {f}")
    print("="*40)
    try:
        with open(f, "r", encoding="utf-8", errors="replace") as fh:
            for i, line in enumerate(fh):
                print(line.strip())
                if i >= 29:  # stop after 30 lines
                    break
    except Exception as e:
        print(f"[ERROR] Could not read {f}: {e}")


In [ ]:
# =========================
# NCAA Monte Carlo (3,000 trials) using your textfiles + arrays
# =========================

# --- Imports (the ones you asked about + a couple standard ones) ---
import os, glob
import pandas as pd
import numpy as np
from io import StringIO

# -------------------------
# 1) Load ALL tables from disk (.txt/.csv) with flexible parsing
# -------------------------
def read_any_table(path):
    """
    Try to read a table with unknown delimiter:
    - pandas automatic sep=None (engine='python')
    - fallback to whitespace
    - fallback to CSV with comma
    Returns DataFrame, or None if unreadable.
    """
    try:
        return pd.read_csv(path, sep=None, engine="python")
    except Exception:
        try:
            return pd.read_csv(path, delim_whitespace=True, engine="python")
        except Exception:
            try:
                return pd.read_csv(path)
            except Exception:
                return None

def load_all_local_tables(search_dirs=(".", "./data", "/content")):
    """
    Search common Colab/local folders for .txt and .csv files
    and return a concatenated DataFrame (row-bind). Missing columns are kept.
    """
    paths = []
    for d in search_dirs:
        if os.path.isdir(d):
            paths.extend(glob.glob(os.path.join(d, "*.txt")))
            paths.extend(glob.glob(os.path.join(d, "*.csv")))

    frames = []
    for p in paths:
        df = read_any_table(p)
        if df is not None and len(df) > 0:
            df["__source_file"] = os.path.basename(p)
            frames.append(df)

    if frames:
        return pd.concat(frames, ignore_index=True, sort=False)
    else:
        return pd.DataFrame()

files_df = load_all_local_tables()

# -------------------------
# 2) Pull in arrays you added (if they exist) e.g., teams_data = [{...}, ...]
# -------------------------
arrays_df = pd.DataFrame()
try:
    # If you defined teams_data earlier in your notebook, we'll use it.
    if isinstance(teams_data, (list, tuple)) and len(teams_data) > 0:
        arrays_df = pd.DataFrame(teams_data)
except NameError:
    pass  # no arrays provided yet; that's fine

# If you created other arrays (e.g., ratings_data, coaches_data), append them here:
for var_name in ["ratings_data", "coaches_data", "roster_data"]:
    try:
        _val = globals().get(var_name, None)
        if isinstance(_val, (list, tuple)) and len(_val) > 0:
            arrays_df = pd.concat([arrays_df, pd.DataFrame(_val)], ignore_index=True, sort=False)
    except Exception:
        pass

# -------------------------
# 3) Combine file tables + arrays, standardize column names
# -------------------------
raw = pd.concat([files_df, arrays_df], ignore_index=True, sort=False)

# Canonical names we’ll try to map to:
# team, conf, FPI, TalentPoints, ReturningStartersPct, QB_Snaps, coach_career_win_pct, OVR, OFF, DEF, QB_AVG
rename_map_candidates = {
    "team": ["team", "Team", "school", "School", "program", "Program"],
    "conf": ["conf", "conference", "Conference", "Conf"],
    "FPI": ["FPI", "fpi", "espn_fpi", "espnFPI"],
    "TalentPoints": ["TalentPoints", "talent_points", "Talent", "Talent_Points", "BlueChipPoints", "blue_chip_points"],
    "ReturningStartersPct": ["ReturningStartersPct", "ReturningStarters%", "ReturningPct", "returning_pct", "Returning_Starters"],
    "QB_Snaps": ["QB_Snaps", "QB Snaps", "qb_snaps", "qb_total_snaps"],
    "coach_career_win_pct": ["coach_career_win_pct", "CoachWinPct", "coach_win_pct", "coachCareerWinPct", "Coach_Career_Win_Pct"],
    "OVR": ["OVR", "Overall", "overall_rating", "overall"],
    "OFF": ["OFF", "Offense", "off_rating", "OFF_Rating"],
    "DEF": ["DEF", "Defense", "def_rating", "DEF_Rating"],
    "QB_AVG": ["QB_AVG", "QB_Avg", "QB_Rating", "qb_overall", "QB"]
}

def smart_rename_columns(df):
    colmap = {}
    lower_cols = {c.lower(): c for c in df.columns}
    for canonical, options in rename_map_candidates.items():
        for opt in options:
            if opt in df.columns:
                colmap[opt] = canonical
                break
            # also try case-insensitive
            opt_low = opt.lower()
            if opt_low in lower_cols:
                colmap[lower_cols[opt_low]] = canonical
                break
    return df.rename(columns=colmap)

if raw.empty:
    raise ValueError("No data found. Ensure your .txt/.csv files are in the working directory or define arrays like `teams_data`.")

raw = smart_rename_columns(raw)

# Keep only relevant columns (others are fine to keep; we’ll ignore them)
wanted_cols = ["team","conf","FPI","TalentPoints","ReturningStartersPct","QB_Snaps","coach_career_win_pct","OVR","OFF","DEF","QB_AVG","__source_file"]
present_cols = [c for c in wanted_cols if c in raw.columns]
data = raw[present_cols].copy()

# Deduplicate by team (keep the last occurrence which is usually the freshest you loaded)
if "team" in data.columns:
    data = data.dropna(subset=["team"]).drop_duplicates(subset=["team"], keep="last")
else:
    raise ValueError("A 'team' column is required in your files/arrays. Please include a team name for each row.")

# Coerce numerics where appropriate
for num_col in ["FPI","TalentPoints","ReturningStartersPct","QB_Snaps","coach_career_win_pct","OVR","OFF","DEF","QB_AVG"]:
    if num_col in data.columns:
        data[num_col] = pd.to_numeric(data[num_col], errors="coerce")

# -------------------------
# 4) Normalize features to avoid one team dominating
#    (min-max to [0,1] per feature that exists)
# -------------------------
feature_weights = {
    "FPI": 0.20,
    "TalentPoints": 0.15,
    "ReturningStartersPct": 0.10,
    "QB_Snaps": 0.10,
    "coach_career_win_pct": 0.10,
    "OVR": 0.15,
    "OFF": 0.10,
    "DEF": 0.07,
    "QB_AVG": 0.03
}
# Keep only weights for columns that actually exist in your data
feature_weights = {k:v for k,v in feature_weights.items() if k in data.columns}

# Re-normalize weights so they sum to 1.0 even if some columns are missing
w_sum = sum(feature_weights.values())
feature_weights = {k: v / w_sum for k,v in feature_weights.items()}

def minmax(series):
    s = series.astype(float)
    lo, hi = s.min(), s.max()
    if pd.isna(lo) or pd.isna(hi) or lo == hi:
        # If constant or NaN, return 0.5 everywhere to avoid bias
        return pd.Series([0.5]*len(s), index=s.index)
    return (s - lo) / (hi - lo)

norm = pd.DataFrame(index=data.index)
for feat in feature_weights.keys():
    norm[f"norm_{feat}"] = minmax(data[feat])

# -------------------------
# 5) Strength score (weighted sum of normalized features)
# -------------------------
def compute_strength(row):
    s = 0.0
    for feat, w in feature_weights.items():
        s += w * row[f"norm_{feat}"]
    return s

data["strength"] = norm.apply(compute_strength, axis=1)

# (Optional) small jitter to break ties without changing overall fairness
np.random.seed(42)
data["strength"] = data["strength"] + np.random.normal(0, 1e-6, size=len(data))

# -------------------------
# 6) Choose teams: use all, or cap at 10 by strength
# -------------------------
USE_TOP_10 = True  # set to False to use every team you loaded
if USE_TOP_10:
    sim_df = data.sort_values("strength", ascending=False).head(10).reset_index(drop=True)
else:
    sim_df = data.reset_index(drop=True)

# -------------------------
# 7) Monte Carlo simulation (3,000 trials), round-robin
# -------------------------
def simulate_game(str_a, str_b):
    # Win prob via Bradley-Terry style model
    p_a = str_a / (str_a + str_b)
    return np.random.rand() < p_a

def monte_carlo_round_robin(df, trials=3000, seed=123):
    rng = np.random.default_rng(seed)
    teams = df["team"].tolist()
    strengths = df["strength"].to_numpy()
    idx = np.arange(len(teams))

    wins = np.zeros(len(teams), dtype=np.int64)

    for _ in range(trials):
        # every unordered pair once per trial
        for i in range(len(teams)):
            for j in range(i+1, len(teams)):
                p_i = strengths[i] / (strengths[i] + strengths[j])
                # sample winner
                if rng.random() < p_i:
                    wins[i] += 1
                else:
                    wins[j] += 1

    total_games = (len(teams) * (len(teams) - 1) // 2) * trials
    # convert to share of all games won; this is not "title odds" but a robust power ranking proxy
    win_share = wins / total_games
    out = pd.DataFrame({"Team": teams, "Win_Share": win_share})
    # also include the normalized strength for transparency
    out["Strength"] = df["strength"].to_numpy()
    return out.sort_values("Win_Share", ascending=False).reset_index(drop=True)

results = monte_carlo_round_robin(sim_df, trials=3000, seed=2025)

# -------------------------
# 8) Display results
# -------------------------
pd.set_option("display.precision", 4)
print("\n=== Monte Carlo Results (3,000 trials) ===")
print(results)

print("\n(Info) Teams included:")
print(sim_df[["team","conf"]].fillna("N/A") if "conf" in sim_df.columns else sim_df[["team"]])

print("\n(Info) Features used & weights after availability check:")
print(feature_weights)

# Optional: write results to CSV for your records
results.to_csv("monte_carlo_results.csv", index=False)
print("\nSaved: monte_carlo_results.csv")


/tmp/ipython-input-3604557381.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, engine="python")
/tmp/ipython-input-3604557381.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, engine="python")
/tmp/ipython-input-3604557381.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, engine="python")
/tmp/ipython-input-3604557381.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(path, delim_whitespace=True, engine="python")
/tmp/ipython-inp


=== Monte Carlo Results (3,000 trials) ===
         Team  Win_Share  Strength
0       Texas     0.1340    0.8004
1     Alabama     0.1239    0.6704
2     Georgia     0.1142    0.5653
3  Ohio State     0.1104    0.5164
4  Penn State     0.1075    0.4891
5     Clemson     0.1017    0.4423
6      Oregon     0.0899    0.3486
7  Notre Dame     0.0800    0.2848
8         LSU     0.0759    0.2776
9       Miami     0.0626    0.2042

(Info) Teams included:
         team     conf
0       Texas      SEC
1     Alabama      SEC
2     Georgia      SEC
3  Ohio State  Big Ten
4  Penn State  Big Ten
5     Clemson      ACC
6      Oregon   Pac-12
7  Notre Dame      Ind
8         LSU      SEC
9       Miami      ACC

(Info) Features used & weights after availability check:
{'FPI': 0.36363636363636365, 'TalentPoints': 0.2727272727272727, 'ReturningStartersPct': 0.18181818181818182, 'QB_Snaps': 0.18181818181818182}

Saved: monte_carlo_results.csv


In [ ]:
# =========================
# NCAA Monte Carlo (3,000 trials) — full pipeline
# =========================

# --- Imports ---
import os, glob
import pandas as pd
import numpy as np
from io import StringIO

# -------------------------
# 1) Load ALL tables from disk (.txt/.csv) with flexible parsing
# -------------------------
def read_any_table(path):
    try:
        return pd.read_csv(path, sep=None, engine="python")
    except Exception:
        try:
            return pd.read_csv(path, delim_whitespace=True, engine="python")
        except Exception:
            try:
                return pd.read_csv(path)
            except Exception:
                return None

def load_all_local_tables(search_dirs=(".", "./data", "/content")):
    paths = []
    for d in search_dirs:
        if os.path.isdir(d):
            paths.extend(glob.glob(os.path.join(d, "*.txt")))
            paths.extend(glob.glob(os.path.join(d, "*.csv")))
    frames = []
    for p in paths:
        df = read_any_table(p)
        if df is not None and len(df) > 0:
            df["__source_file"] = os.path.basename(p)
            frames.append(df)
    return pd.concat(frames, ignore_index=True, sort=False) if frames else pd.DataFrame()

files_df = load_all_local_tables()

# -------------------------
# 2) Pull in arrays you added (if they exist) e.g., teams_data = [{...}, ...]
# -------------------------
arrays_df = pd.DataFrame()
for var_name in ["teams_data", "ratings_data", "coaches_data", "roster_data"]:
    try:
        _val = globals().get(var_name, None)
        if isinstance(_val, (list, tuple)) and len(_val) > 0:
            arrays_df = pd.concat([arrays_df, pd.DataFrame(_val)], ignore_index=True, sort=False)
    except Exception:
        pass

# -------------------------
# 3) Combine & standardize columns
# -------------------------
raw = pd.concat([files_df, arrays_df], ignore_index=True, sort=False)

rename_map_candidates = {
    "team": ["team","Team","school","School","program","Program"],
    "conf": ["conf","conference","Conference","Conf"],
    "FPI": ["FPI","fpi","espn_fpi","espnFPI"],
    "TalentPoints": ["TalentPoints","talent_points","Talent","Talent_Points","BlueChipPoints","blue_chip_points"],
    "ReturningStartersPct": ["ReturningStartersPct","ReturningStarters%","ReturningPct","returning_pct","Returning_Starters"],
    "QB_Snaps": ["QB_Snaps","QB Snaps","qb_snaps","qb_total_snaps"],
    "coach_career_win_pct": ["coach_career_win_pct","CoachWinPct","coach_win_pct","coachCareerWinPct","Coach_Career_Win_Pct"],
    "OVR": ["OVR","Overall","overall_rating","overall"],
    "OFF": ["OFF","Offense","off_rating","OFF_Rating"],
    "DEF": ["DEF","Defense","def_rating","DEF_Rating"],
    "QB_AVG": ["QB_AVG","QB_Avg","QB_Rating","qb_overall","QB"],
    "Top3_OFF_AVG": ["Top3_OFF_AVG","Top3_Off_Avg","TopOff3","Top3_Offense"],
    "Top3_DEF_AVG": ["Top3_DEF_AVG","Top3_Def_Avg","TopDef3","Top3_Defense"],
}

def smart_rename_columns(df):
    colmap = {}
    lower_cols = {c.lower(): c for c in df.columns}
    for canonical, options in rename_map_candidates.items():
        for opt in options:
            if opt in df.columns:
                colmap[opt] = canonical
                break
            opt_low = opt.lower()
            if opt_low in lower_cols:
                colmap[lower_cols[opt_low]] = canonical
                break
    return df.rename(columns=colmap)

if raw.empty:
    raise ValueError("No data found. Put your .txt/.csv files in the working dir OR define arrays like `teams_data`.")

raw = smart_rename_columns(raw)

wanted_cols = ["team","conf","FPI","TalentPoints","ReturningStartersPct","QB_Snaps",
               "coach_career_win_pct","OVR","OFF","DEF","QB_AVG","Top3_OFF_AVG","Top3_DEF_AVG","__source_file"]
present_cols = [c for c in wanted_cols if c in raw.columns]
data = raw[present_cols].copy()

if "team" not in data.columns:
    raise ValueError("A 'team' column is required in your data.")

# Deduplicate by team (keep latest)
data = data.dropna(subset=["team"]).drop_duplicates(subset=["team"], keep="last")

# Coerce numerics
for num in ["FPI","TalentPoints","ReturningStartersPct","QB_Snaps","coach_career_win_pct",
            "OVR","OFF","DEF","QB_AVG","Top3_OFF_AVG","Top3_DEF_AVG"]:
    if num in data.columns:
        data[num] = pd.to_numeric(data[num], errors="coerce")

# -------------------------
# 4) Ensure missing rating columns exist, then fill for ALL 10 teams
# -------------------------
for col in ["coach_career_win_pct","OVR","OFF","DEF","QB_AVG","Top3_OFF_AVG","Top3_DEF_AVG"]:
    if col not in data.columns:
        data[col] = np.nan

# Ratings for your 10 teams (edit if your team names differ)
ratings_updates = {
    "Texas":       {"coach_career_win_pct": 75, "OVR": 95, "OFF": 94, "DEF": 93, "QB_AVG": 92, "Top3_OFF_AVG": 96, "Top3_DEF_AVG": 94},
    "Alabama":     {"coach_career_win_pct": 80, "OVR": 94, "OFF": 93, "DEF": 94, "QB_AVG": 90, "Top3_OFF_AVG": 95, "Top3_DEF_AVG": 95},
    "Georgia":     {"coach_career_win_pct": 78, "OVR": 94, "OFF": 92, "DEF": 95, "QB_AVG": 91, "Top3_OFF_AVG": 94, "Top3_DEF_AVG": 96},
    "Ohio State":  {"coach_career_win_pct": 77, "OVR": 93, "OFF": 93, "DEF": 92, "QB_AVG": 90, "Top3_OFF_AVG": 95, "Top3_DEF_AVG": 93},
    "Penn State":  {"coach_career_win_pct": 70, "OVR": 91, "OFF": 90, "DEF": 91, "QB_AVG": 88, "Top3_OFF_AVG": 92, "Top3_DEF_AVG": 91},
    "Clemson":     {"coach_career_win_pct": 73, "OVR": 90, "OFF": 90, "DEF": 89, "QB_AVG": 87, "Top3_OFF_AVG": 91, "Top3_DEF_AVG": 90},
    "Oregon":      {"coach_career_win_pct": 72, "OVR": 91, "OFF": 92, "DEF": 88, "QB_AVG": 89, "Top3_OFF_AVG": 93, "Top3_DEF_AVG": 90},
    "Notre Dame":  {"coach_career_win_pct": 74, "OVR": 90, "OFF": 89, "DEF": 89, "QB_AVG": 88, "Top3_OFF_AVG": 90, "Top3_DEF_AVG": 91},
    "LSU":         {"coach_career_win_pct": 71, "OVR": 91, "OFF": 91, "DEF": 90, "QB_AVG": 88, "Top3_OFF_AVG": 93, "Top3_DEF_AVG": 92},
    "Miami":       {"coach_career_win_pct": 69, "OVR": 88, "OFF": 87, "DEF": 88, "QB_AVG": 86, "Top3_OFF_AVG": 89, "Top3_DEF_AVG": 88},
}

# Apply where team names match
for team, vals in ratings_updates.items():
    mask = data['team'].str.strip().str.casefold() == team.casefold()
    for col, val in vals.items():
        data.loc[mask, col] = data.loc[mask, col].fillna(val)

# For any remaining NaNs in these columns, backfill with column medians (prevents dropping teams)
for col in ["coach_career_win_pct","OVR","OFF","DEF","QB_AVG","Top3_OFF_AVG","Top3_DEF_AVG"]:
    if col in data.columns:
        if data[col].isna().any():
            data[col] = data[col].fillna(data[col].median())

# -------------------------
# 5) Normalize features & compute strength
# -------------------------
feature_weights = {
    # team quality + roster/talent
    "FPI": 0.12,
    "TalentPoints": 0.10,
    "ReturningStartersPct": 0.08,
    "QB_Snaps": 0.07,
    # coaching + EA team ratings
    "coach_career_win_pct": 0.10,
    "OVR": 0.12,
    "OFF": 0.08,
    "DEF": 0.08,
    "QB_AVG": 0.07,
    # NEW: top-3 impact players
    "Top3_OFF_AVG": 0.09,
    "Top3_DEF_AVG": 0.09,
}
# keep only existing columns, then renormalize to sum=1
feature_weights = {k:v for k,v in feature_weights.items() if k in data.columns}
w_sum = sum(feature_weights.values())
feature_weights = {k: v / w_sum for k,v in feature_weights.items()}

def minmax(series):
    s = pd.to_numeric(series, errors='coerce').astype(float)
    lo, hi = s.min(), s.max()
    if pd.isna(lo) or pd.isna(hi) or lo == hi:
        return pd.Series([0.5]*len(s), index=s.index)
    return (s - lo) / (hi - lo)

norm = pd.DataFrame(index=data.index)
for feat in feature_weights.keys():
    norm[f"norm_{feat}"] = minmax(data[feat])

def compute_strength(row):
    return sum(feature_weights[feat] * row[f"norm_{feat}"] for feat in feature_weights.keys())

np.random.seed(42)
data["strength"] = norm.apply(compute_strength, axis=1) + np.random.normal(0, 1e-6, size=len(data))

# -------------------------
# 6) Choose teams (top 10 by strength)
# -------------------------
USE_TOP_10 = True
sim_df = data.sort_values("strength", ascending=False).head(10).reset_index(drop=True) if USE_TOP_10 else data.reset_index(drop=True)

# -------------------------
# 7) Monte Carlo (3,000 trials), round-robin
# -------------------------
def monte_carlo_round_robin(df, trials=3000, seed=2025):
    rng = np.random.default_rng(seed)
    teams = df["team"].tolist()
    strengths = df["strength"].to_numpy()
    n = len(teams)
    wins = np.zeros(n, dtype=np.int64)

    for _ in range(trials):
        for i in range(n):
            for j in range(i+1, n):
                p_i = strengths[i] / (strengths[i] + strengths[j])
                if rng.random() < p_i:
                    wins[i] += 1
                else:
                    wins[j] += 1

    total_games = (n * (n - 1) // 2) * trials
    win_share = wins / total_games
    results = pd.DataFrame({"Team": teams, "Win_Share": win_share, "Strength": strengths})
    return results.sort_values("Win_Share", ascending=False).reset_index(drop=True)

# -------------------------
# 8) Run & print
# -------------------------
results = monte_carlo_round_robin(sim_df, trials=3000, seed=2025)

pd.set_option("display.precision", 4)
print("\n=== Monte Carlo Results (3,000 trials, all variables incl. Top3 players) ===")
print(results)

print("\n(Info) Teams included:")
cols = ["team","conf"] if "conf" in sim_df.columns else ["team"]
print(sim_df[cols])

print("\n(Info) Features used & final weights:")
print(feature_weights)

# Optional: save
results.to_csv("monte_carlo_results_all_vars.csv", index=False)
print("\nSaved: monte_carlo_results_all_vars.csv")



In [ ]:
import numpy as np
import pandas as pd

# --- Monte Carlo WITHOUT round-robin ---
# Each season: every team plays G games against randomly chosen opponents
# Home-field adds a small bump to the home team.
def monte_carlo_random_schedule(df, trials=3000, games_per_team=12, home_field_adv=0.03, seed=2025):
    rng = np.random.default_rng(seed)
    teams = df["team"].tolist()
    strengths = df["strength"].to_numpy()
    n = len(teams)

    # trackers across all trials
    avg_winrate = np.zeros(n, dtype=float)

    for _ in range(trials):
        # per-season counters
        wins = np.zeros(n, dtype=int)
        remaining = np.full(n, games_per_team, dtype=int)

        # schedule games until every team has played `games_per_team`
        while remaining.sum() > 0:
            # pick team A that still needs a game
            i_candidates = np.where(remaining > 0)[0]
            i = rng.choice(i_candidates)

            # pick opponent B that still needs a game (not i)
            j_candidates = i_candidates[i_candidates != i]
            if j_candidates.size == 0:
                # if no opponent available due to odd scheduling (rare), skip
                remaining[i] = 0
                continue
            j = rng.choice(j_candidates)

            # random home team
            home_is_i = rng.random() < 0.5

            # base win prob using Bradley–Terry style
            p_i = strengths[i] / (strengths[i] + strengths[j])

            # apply home field advantage
            if home_is_i:
                p_i = np.clip(p_i + home_field_adv, 0.01, 0.99)
            else:
                p_i = np.clip(p_i - home_field_adv, 0.01, 0.99)

            # play game
            if rng.random() < p_i:
                wins[i] += 1
            else:
                wins[j] += 1

            # both teams used one game slot
            remaining[i] -= 1
            remaining[j] -= 1

        # season win rate for each team
        winrate = wins / games_per_team
        avg_winrate += winrate

    # average across trials
    avg_winrate /= trials

    results = pd.DataFrame({
        "Team": teams,
        "Avg_WinRate": avg_winrate,
        "Strength": strengths
    }).sort_values("Avg_WinRate", ascending=False).reset_index(drop=True)

    return results

# ---- Run it (no round-robin) ----
results_no_rr = monte_carlo_random_schedule(data, trials=3000, games_per_team=12, home_field_adv=0.03, seed=2025)
print("\n=== Monte Carlo Results (3,000 seasons, RANDOM schedules, no round-robin) ===")
print(results_no_rr)



=== Monte Carlo Results (3,000 seasons, RANDOM schedules, no round-robin) ===
         Team  Avg_WinRate  Strength
0       Texas       0.6581    0.8276
1     Alabama       0.6482    0.7939
2     Georgia       0.6292    0.7427
3  Ohio State       0.5973    0.6365
4  Penn State       0.4839    0.4129
5      Oregon       0.4644    0.3811
6         LSU       0.4546    0.3687
7     Clemson       0.4479    0.3480
8  Notre Dame       0.4060    0.2975
9       Miami       0.1465    0.0830


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_rankings(df, metric_col, title):
    df_plot = df.copy().sort_values(metric_col, ascending=True)
    plt.figure(figsize=(9, 6))
    plt.barh(df_plot["Team"], df_plot[metric_col])
    plt.xlabel(metric_col)
    plt.title(title)
    for i, v in enumerate(df_plot[metric_col]):
        plt.text(v, i, f" {v:.3f}", va="center")
    plt.tight_layout()
    plt.show()

# Choose the results you produced:
if "results_no_rr" in globals():
    plot_rankings(results_no_rr, "Avg_WinRate", "Average Win Rate (3,000 seasons, random schedules)")
elif "results" in globals():
    plot_rankings(results, "Win_Share", "Win Share (3,000 trials, round-robin)")


In [ ]:
week1_results_model = [
    {"winner": "Ohio State", "winner_rank": 3, "loser": "Texas", "loser_rank": 1, "score": "14-7"},
    {"winner": "Penn State", "winner_rank": 2, "loser": "Nevada", "loser_rank": None, "score": "46-11"},
    {"winner": "LSU", "winner_rank": 9, "loser": "Clemson", "loser_rank": 4, "score": "17-10"},
    {"winner": "Georgia", "winner_rank": 5, "loser": "Marshall", "loser_rank": None, "score": "45-7"},
    {"winner": "Oregon", "winner_rank": 7, "loser": "Montana State", "loser_rank": None, "score": "59-13"},
    {"winner": "Alabama", "winner_rank": 8, "loser": "Florida State", "loser_rank": None, "score": "31-17"},
    # Miami and Notre Dame didn't have games listed here
]


In [ ]:
# ===================== WEEK 1 RESULTS -> RATING UPDATE -> RE-RUN TRIALS =====================

# --- 0) Week 1 results: only teams in your 10-team model ---
week1_results_model = [
    {"winner": "Ohio State", "winner_rank": 3, "loser": "Texas", "loser_rank": 1, "score": "14-7"},
    {"winner": "Penn State", "winner_rank": 2, "loser": "Nevada", "loser_rank": None, "score": "46-11"},
    {"winner": "LSU", "winner_rank": 9, "loser": "Clemson", "loser_rank": 4, "score": "17-10"},
    {"winner": "Georgia", "winner_rank": 5, "loser": "Marshall", "loser_rank": None, "score": "45-7"},
    {"winner": "Oregon", "winner_rank": 7, "loser": "Montana State", "loser_rank": None, "score": "59-13"},
    {"winner": "Alabama", "winner_rank": 8, "loser": "Florida State", "loser_rank": None, "score": "31-17"},
    # Miami and Notre Dame not listed in your Week 1 results
]

# --- 1) Helper: get rating values safely ---
def _get_one(series):
    """Return a single scalar from a 1-row series (.item() is cleaner than float(series))."""
    return series.item()

# --- 2) Win probability (same shape as earlier, with a softer scale) ---
SCALE = 6.0
def win_prob(r_i, r_j, scale=SCALE):
    return 1.0 / (1.0 + np.exp(-(r_i - r_j) / scale))

# --- 3) Apply Elo-style post-game updates (gentle; includes margin-of-victory damping) ---
def apply_results_update(df_in, results, K=0.9, cap_mov=28):
    """
    df_in: DataFrame with columns ['team','rating'] (your fair rating)
    results: list of dicts with keys winner, loser, score ("A-B")
    K: learning rate (0.6–1.2 reasonable). Lower => smaller moves.
    cap_mov: cap on absolute point differential for stability.
    """
    df_upd = df_in.copy()
    df_upd = df_upd.set_index("team")

    for g in results:
        w, l = g["winner"], g["loser"]
        # skip games where the opponent is not in the model (e.g., Nevada, Marshall)
        if w not in df_upd.index or l not in df_upd.index:
            continue

        # parse margin of victory
        try:
            a, b = g["score"].split("-")
            mov = max(0, min(cap_mov, abs(int(a) - int(b))))
        except Exception:
            mov = 0  # if score not parseable, ignore MOV

        # ratings before update
        rw = df_upd.at[w, "rating"]
        rl = df_upd.at[l, "rating"]

        # expected win prob for winner vs loser
        p_exp = win_prob(rw, rl, scale=SCALE)

        # margin multiplier: log(1+mov) keeps big blowouts from exploding updates
        mov_mult = np.log1p(mov) / np.log(8)  # ~1.0 around 7–8 pts, <1 for small wins, >1 for big wins
        mov_mult = float(np.clip(mov_mult, 0.5, 1.5))

        # update (winner moves up by (1 - p), loser down by same amount)
        delta = K * mov_mult * (1.0 - p_exp)
        df_upd.at[w, "rating"] = rw + delta
        df_upd.at[l, "rating"] = rl - delta

    # re-standardize so the field mean=0, std=1 (keeps compatibility with your sim scale)
    mu = df_upd["rating"].mean()
    sd = df_upd["rating"].std(ddof=0) + 1e-9
    df_upd["rating"] = (df_upd["rating"] - mu) / sd

    return df_upd.reset_index()

df_updated = apply_results_update(df[["team", "rating"]], week1_results_model, K=0.9, cap_mov=28)
print("Ratings updated with Week 1 results (top 10):")
print(df_updated.sort_values("rating", ascending=False).head(10))

# --- 4) Minimal simulation utilities (use these if you don't already have them defined) ---
def simulate_season(n_trials, table, scale=SCALE, seed=None):
    """
    Round-robin neutral-site season repeated n_trials times.
    Returns average win share per team.
    """
    rng = np.random.default_rng(seed)
    teams = table["team"].tolist()
    rmap = dict(zip(table["team"], table["rating"]))

    wins = dict((t, 0.0) for t in teams)
    games_per_team = len(teams) - 1

    for _ in range(n_trials):
        for i in range(len(teams)):
            for j in range(i + 1, len(teams)):
                ti, tj = teams[i], teams[j]
                p = win_prob(rmap[ti], rmap[tj], scale=scale)
                if rng.random() < p:
                    wins[ti] += 1
                else:
                    wins[tj] += 1

    # convert to average share
    share = {t: wins[t] / (n_trials * games_per_team) for t in teams}
    return pd.Series(share).sort_values(ascending=False)

def run_trials(table, n_trials=2400, seed=2025):
    """Return summary table with average win share and simple 'champ' frequency (most wins each trial)."""
    rng = np.random.default_rng(seed)
    teams = table["team"].tolist()
    rmap = dict(zip(table["team"], table["rating"]))

    champ_counts = dict((t, 0) for t in teams)
    games_per_team = len(teams) - 1
    wins_accum = dict((t, 0.0) for t in teams)

    for t in range(n_trials):
        # simulate one full round-robin season
        wins = dict((tname, 0) for tname in teams)
        for i in range(len(teams)):
            for j in range(i + 1, len(teams)):
                ti, tj = teams[i], teams[j]
                p = win_prob(rmap[ti], rmap[tj], scale=SCALE)
                if rng.random() < p:
                    wins[ti] += 1
                else:
                    wins[tj] += 1
        # accumulate
        for k in teams:
            wins_accum[k] += wins[k]
        # champion = max wins (break ties randomly)
        max_w = max(wins.values())
        tied = [k for k, v in wins.items() if v == max_w]
        champ = rng.choice(tied)
        champ_counts[champ] += 1

    avg_share = {k: wins_accum[k] / (n_trials * games_per_team) for k in teams}
    champ_prob = {k: champ_counts[k] / n_trials for k in teams}

    out = pd.DataFrame({
        "team": teams,
        "avg_win_share": [avg_share[k] for k in teams],
        "champ_prob": [champ_prob[k] for k in teams],
        "rating": [rmap[k] for k in teams],
    }).sort_values(["champ_prob", "avg_win_share"], ascending=False, ignore_index=True)

    return out

# --- 5) RUN: Monte-Carlo on updated ratings ---
results_after_w1 = run_trials(df_updated, n_trials=2400, seed=2026)
print("\nMonte Carlo (after Week 1 updates) — top 10:")
print(results_after_w1.head(10))


In [ ]:
# If you already have df_updated or a merged table with rating, rename it back to df
# Replace df_updated with your actual DataFrame that contains "team" and "rating"
df = df_updated  # <-- Or assign your merged DataFrame here


In [ ]:
df_updated = apply_results_update(df[["team", "rating"]], week1_results_model, K=0.9, cap_mov=28)
print("Ratings updated with Week 1 results (top 10):")
print(df_updated.sort_values("rating", ascending=False).head(10))


In [ ]:
# ================= FULL NCAA MODEL RESET & WEEK 1 SIM =================

import pandas as pd
import numpy as np

# 1) BASE DATA: from your earlier array
teams_data = [
    {"team": "Texas", "conf": "SEC", "FPI": 24.5, "TalentRank": 4,
     "TalentPoints": 973.54, "ReturningStartersPct": 55.0, "QB_Snaps": 260},
    {"team": "Alabama", "conf": "SEC", "FPI": 20.7, "TalentRank": 2,
     "TalentPoints": 993.55, "ReturningStartersPct": 63.6, "QB_Snaps": 60},
    {"team": "Ohio State", "conf": "Big Ten", "FPI": 20.5, "TalentRank": 3,
     "TalentPoints": 973.69, "ReturningStartersPct": 50.0, "QB_Snaps": 180},
    {"team": "Georgia", "conf": "SEC", "FPI": 19.8, "TalentRank": 1,
     "TalentPoints": 999.9, "ReturningStartersPct": 58.0, "QB_Snaps": 40},
    {"team": "Penn State", "conf": "Big Ten", "FPI": 18.2, "TalentRank": 6,
     "TalentPoints": 950.1, "ReturningStartersPct": 62.0, "QB_Snaps": 150},
    {"team": "Clemson", "conf": "ACC", "FPI": 17.0, "TalentRank": 7,
     "TalentPoints": 940.0, "ReturningStartersPct": 60.0, "QB_Snaps": 220},
    {"team": "Notre Dame", "conf": "Ind", "FPI": 16.5, "TalentRank": 8,
     "TalentPoints": 930.0, "ReturningStartersPct": 61.0, "QB_Snaps": 75},
    {"team": "Oregon", "conf": "Pac-12", "FPI": 15.8, "TalentRank": 5,
     "TalentPoints": 960.0, "ReturningStartersPct": 59.0, "QB_Snaps": 100},
    {"team": "LSU", "conf": "SEC", "FPI": 15.2, "TalentRank": 9,
     "TalentPoints": 925.0, "ReturningStartersPct": 57.0, "QB_Snaps": 210},
    {"team": "Miami", "conf": "ACC", "FPI": 14.9, "TalentRank": 10,
     "TalentPoints": 915.0, "ReturningStartersPct": 56.0, "QB_Snaps": 190},
]
df = pd.DataFrame(teams_data)

# 2) FAIR RATING CALC
def robust_z(s: pd.Series):
    med = s.median()
    iqr = s.quantile(0.75) - s.quantile(0.25)
    if iqr == 0:
        std = s.std(ddof=0) + 1e-9
        return (s - med) / std
    return (s - med) / (iqr + 1e-9)

numeric_cols = ["FPI","TalentPoints","ReturningStartersPct","QB_Snaps"]
for c in numeric_cols:
    df[c+"_rz"] = robust_z(df[c]).clip(-1.5, 1.5)

weights = {
    "FPI_rz": 0.25, "TalentPoints_rz": 0.20,
    "ReturningStartersPct_rz": 0.15, "QB_Snaps_rz": 0.10
}
total_w = sum(weights.values())
weights = {k: v/total_w for k,v in weights.items()}

df["feature_score"] = sum(df[k]*w for k,w in weights.items())
df["rating_fair_raw"] = 0.7 * df["feature_score"]
mu, sd = df["rating_fair_raw"].mean(), df["rating_fair_raw"].std(ddof=0)+1e-9
df["rating"] = (df["rating_fair_raw"] - mu)/sd

# 3) WEEK 1 RESULTS
week1_results_model = [
    {"winner": "Ohio State", "loser": "Texas", "score": "14-7"},
    {"winner": "Penn State", "loser": "Nevada", "score": "46-11"},
    {"winner": "LSU", "loser": "Clemson", "score": "17-10"},
    {"winner": "Georgia", "loser": "Marshall", "score": "45-7"},
    {"winner": "Oregon", "loser": "Montana State", "score": "59-13"},
    {"winner": "Alabama", "loser": "Florida State", "score": "31-17"},
]

SCALE = 6.0
def win_prob(r1, r2, scale=SCALE):
    return 1 / (1 + np.exp(-(r1 - r2)/scale))

def apply_results_update(df_in, results, K=0.9):
    df_upd = df_in.set_index("team").copy()
    for g in results:
        w, l = g["winner"], g["loser"]
        if w not in df_upd.index or l not in df_upd.index:  # skip non-model teams
            continue
        rw, rl = df_upd.at[w, "rating"], df_upd.at[l, "rating"]
        p_exp = win_prob(rw, rl)
        delta = K * (1 - p_exp)
        df_upd.at[w, "rating"] = rw + delta
        df_upd.at[l, "rating"] = rl - delta
    # normalize ratings
    mu, sd = df_upd["rating"].mean(), df_upd["rating"].std(ddof=0)+1e-9
    df_upd["rating"] = (df_upd["rating"] - mu)/sd
    return df_upd.reset_index()

df_updated = apply_results_update(df, week1_results_model)

# 4) SIMULATION
def run_trials(table, n_trials=1000):
    teams = table["team"].tolist()
    rmap = dict(zip(table["team"], table["rating"]))
    wins = {t: 0 for t in teams}
    games_per_team = len(teams)-1
    rng = np.random.default_rng()
    for _ in range(n_trials):
        for i in range(len(teams)):
            for j in range(i+1,len(teams)):
                ti, tj = teams[i], teams[j]
                p = win_prob(rmap[ti], rmap[tj])
                if rng.random() < p:
                    wins[ti]+=1
                else:
                    wins[tj]+=1
    share = {t: wins[t]/(n_trials*games_per_team) for t in teams}
    return pd.DataFrame({"team":teams,"avg_win_share":[share[t] for t in teams]}).sort_values("avg_win_share",ascending=False)

# 5) OUTPUT
print("Updated ratings after Week 1:")
print(df_updated.sort_values("rating",ascending=False))

print("\nSimulation results (avg win share):")
print(run_trials(df_updated, n_trials=2000))


Updated ratings after Week 1:
         team     conf   FPI  TalentRank  TalentPoints  ReturningStartersPct  \
1     Alabama      SEC  20.7           2        993.55                  63.6   
0       Texas      SEC  24.5           4        973.54                  55.0   
3     Georgia      SEC  19.8           1        999.90                  58.0   
2  Ohio State  Big Ten  20.5           3        973.69                  50.0   
4  Penn State  Big Ten  18.2           6        950.10                  62.0   
7      Oregon   Pac-12  15.8           5        960.00                  59.0   
5     Clemson      ACC  17.0           7        940.00                  60.0   
8         LSU      SEC  15.2           9        925.00                  57.0   
6  Notre Dame      Ind  16.5           8        930.00                  61.0   
9       Miami      ACC  14.9          10        915.00                  56.0   

   QB_Snaps    FPI_rz  TalentPoints_rz  ReturningStartersPct_rz  QB_Snaps_rz  \
1        

🏈 Rankings with QB Snaps heavily weighted:
         team  rating_qb
0       Texas   2.338406
5     Clemson   0.736919
2  Ohio State   0.496955
4  Penn State   0.195302
8         LSU  -0.033723
1     Alabama  -0.087250
9       Miami  -0.534021
3     Georgia  -0.805251
7      Oregon  -0.930683
6  Notre Dame  -1.376652


so who is